In [121]:
import numpy as np

In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

In [257]:
from sklearn.tree import DecisionTreeRegressor

In [15]:
import pandas as pd

In [19]:
df = pd.read_csv('../data/raw_districtwise_yield_data.csv')

In [21]:
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246091 entries, 0 to 246090
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   State_Name     246091 non-null  object 
 1   District_Name  246091 non-null  object 
 2   Crop_Year      246091 non-null  int64  
 3   Season         246091 non-null  object 
 4   Crop           246091 non-null  object 
 5   Area           246091 non-null  float64
 6   Production     242361 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 13.1+ MB


In [25]:
df

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0
...,...,...,...,...,...,...,...
246086,West Bengal,PURULIA,2014,Summer,Rice,306.0,801.0
246087,West Bengal,PURULIA,2014,Summer,Sesamum,627.0,463.0
246088,West Bengal,PURULIA,2014,Whole Year,Sugarcane,324.0,16250.0
246089,West Bengal,PURULIA,2014,Winter,Rice,279151.0,597899.0


In [33]:
df['Yield'] = df['Production'] / df['Area']

In [39]:
# Encode categorical variables
label_encoders = {}
for column in ['State_Name', 'District_Name', 'Season', 'Crop']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [61]:
# Define features and target
X = df[['State_Name', 'District_Name', 'Crop_Year', 'Season', 'Crop', 'Area']]
y = df['Yield']

In [63]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [65]:
df.isnull().sum()

State_Name       0
District_Name    0
Crop_Year        0
Season           0
Crop             0
Area             0
Production       0
Yield            0
dtype: int64

In [59]:
df = df.dropna()

In [259]:
dtmodel = DecisionTreeRegressor(random_state=42)

In [261]:
dtmodel.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [263]:
y_pred = dtmodel.predict(X_test)

In [265]:
mse = mean_squared_error(y_test, y_pred)
mse

204495.59109574454

In [267]:
mae = mean_absolute_error(y_test, y_pred)
mae

13.765570024257073

In [269]:
X_test

,State_Name,District_Name,Crop_Year,Season,Crop,Area
51163,6,272,2007,2,119,5467.0
55141,6,476,2012,2,92,830.0
104122,16,65,2012,4,87,9.0
161230,23,643,2010,2,63,13.0
229232,30,549,2002,2,59,117.0
...,...,...,...,...,...,...
181723,27,336,2005,1,25,619.0
140533,19,496,2013,4,87,30.0
113952,16,309,2008,1,25,134.0
148435,22,61,2005,0,43,17956.0


In [307]:
singlerow = np.array([22, 61, 2005, 0, 43, 17956.0]).reshape(1, -1)

In [309]:
prediction = dtmodel.predict(singlerow)

C:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [311]:
prediction

array([0.86599765])

In [315]:
row = df[df['Area'] == 17956.0]
row

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Yield
87272,14,192,1998,1,59,17956.0,64872.0,3.612831
148435,22,61,2005,0,43,17956.0,16070.6,0.894999


In [317]:
pickle.dump(dtmodel, open('final_tree_model.pkl','wb'))